In [2]:
import numpy as np 
import pandas as pd
import requests
from groq import Groq
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
import math
from openai import OpenAI
import openai
import time

In [16]:
keys =[]
date_ranges = [
    ("2010-01-01", "2016-01-01", "cA"),
    ("2016-01-01", "2017-01-01", "c16"),
    ("2017-01-01", "2018-01-01", "c17"),
    ("2018-01-01", "2019-01-01", "c18"),
    ("2019-01-01", "2019-10-01", "c19"),
    ("2020-01-01", "2021-01-01", "cc20"),
    ("2021-01-01", "2022-01-01", "cc21"),
    ("2022-01-01", "2023-01-01", "cc22"),
    ("2023-01-01", "2024-01-01", "c23"),
    ("2024-01-01", "2025-01-01", "c24")
]

def filter (start_date, end_date, label):
    globals()[label] = contracts_test[
        (contracts_test["Start Date"] >= pd.Timestamp(start_date)) & 
        (contracts_test["Start Date"] < pd.Timestamp(end_date))
    ].reset_index(drop=True, inplace=False)
    keys.append(label)
    
for start, end, label in date_ranges:
    filter(start, end, label)

In [7]:
keys

['cA', 'c16', 'c17', 'c18', 'c19', 'cc20', 'cc21', 'cc22', 'c23', 'c24']

In [8]:
for key in keys:
    print(globals().get(key).shape)

(5909, 21)
(1235, 21)
(1598, 21)
(1996, 21)
(1698, 21)
(2206, 21)
(2048, 21)
(1894, 21)
(1739, 21)
(873, 21)


In [9]:
contractColumns = contracts_test.columns
progressPoints = {0.25: "25% done", 0.5: "50% done", 0.75: "75% done"}

apiCalls =0

In [ ]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(getPublicContractsGPT, key) for key in keys[5:8]]

    for future in futures:
        try:
            future.result()  
        except Exception as e:
            print(f"An error occurred: {e}")

print("All contracts processed.")

started processing cc20started processing cc21

started processing cc22
Rate limit exceeded
Retrying after 0.85 seconds...
Rate limit exceeded
Retrying after 2.05 seconds...
Rate limit exceeded
Retrying after 0.19 seconds...
Rate limit exceeded
Retrying after 4.38 seconds...
Rate limit exceeded
Retrying after 4.38 seconds...
Rate limit exceeded
Retrying after 0.00 seconds...
Rate limit exceeded
Retrying after 4.41 seconds...
Rate limit exceeded
Retrying after 0.23 seconds...
Rate limit exceeded
Retrying after 0.25 seconds...
Rate limit exceeded
Retrying after 0.23 seconds...
Rate limit exceeded
Retrying after 4.37 seconds...
Rate limit exceeded
Retrying after 0.23 seconds...
Rate limit exceeded
Retrying after 0.24 seconds...
Rate limit exceeded
Retrying after 0.01 seconds...
Rate limit exceeded
Retrying after 4.38 seconds...
Rate limit exceeded
Retrying after 3.52 seconds...
Rate limit exceeded
Retrying after 4.37 seconds...
Rate limit exceeded
Retrying after 0.02 seconds...
Rate limit

In [121]:
grouped_pCC22 = {recipient: group for recipient, group in pCC22.groupby('Recipient Name')}
sorted_keys = sorted(grouped_pCC22.keys(), key=lambda x: len(grouped_pCC22[x]), reverse=True)
sorted_keys[:5]

['BA', 'LMT', 'RTN', 'LDOS', 'LHX']

In [156]:
def getHistoric(tik, interval, month):
    global keyCalls, currentKey
    historic = requests.get(alphV, params=setParams(tik, interval, month))
    key = f"Time Series ({interval})"
    if key not in historic.json():
        return pd.DataFrame() 
    else: 
        df = pd.DataFrame.from_dict(historic.json()[key]).transpose().reset_index().rename(columns={'index':'date'})
        df.columns = df.columns.str.replace(r'^\d+\.\s*', '', regex=True)
        df['date'] = pd.to_datetime(df['date'])
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df.sort_values(by='date', ascending= True).reset_index(drop=True)

In [97]:
def getYTDData(tik, interval, contractDate):
    date = datetime.strptime(contractDate, "%Y-%m-%d")
    
    months = []
    for month_offset in range(-1, 13):
        month_date = (date.replace(day=1) + pd.DateOffset(months=month_offset))
        months.append(month_date.strftime("%Y-%m"))
    
    data_frames = []
    for month in months:
        df = getHistoric(tik, interval, month)
        if not df.empty:
            data_frames.append(df)
    
    if data_frames:
        return pd.concat(data_frames, ignore_index=True, sort=False)
    else:
        return pd.DataFrame()

In [161]:
lmTDf = grouped_pCC22["LMT"]
rtxDf = grouped_pCC22["RTN"]
ldosDf = grouped_pCC22["LDOS"]
lhxDf = grouped_pCC22["LHX"]
baDf = grouped_pCC22["BA"]

In [124]:
lmtINTD = getYTDData("LMT", "60min", "2022-01-01")

API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math

graph = 2022
def genGraph(tik, lmtINTD, lmtDf):
    global graph
    
    lmtINTD['date'] = pd.to_datetime(lmtINTD['date'])  
    lmtINTD_2022 = lmtINTD[lmtINTD['date'].dt.year == 2022]

    plt.figure(figsize=(20, 10))
    
    plt.plot(lmtINTD_2022['date'], lmtINTD_2022['close'], label='Close Price', color='blue')

    lmtDf['Start Date'] = pd.to_datetime(lmtDf['Start Date'])  
    contract_dates = lmtDf[lmtDf['Start Date'].dt.year == 2022]['Start Date'].dt.date.unique()  

    for date in contract_dates:
        contract_close_price = lmtINTD_2022[lmtINTD_2022['date'].dt.date == date]['close'].iloc[0] if not lmtINTD_2022[lmtINTD_2022['date'].dt.date == date].empty else None
        if contract_close_price is not None:
            plt.scatter(date, contract_close_price, color='red', label=f'Contract on {date}')

    x_min = lmtINTD_2022['date'].min().strftime("%Y-%m-%d")
    x_max = lmtINTD_2022['date'].max().strftime("%Y-%m-%d")
    y_min = lmtINTD_2022['close'].min()
    y_max = lmtINTD_2022['close'].max()
    y_ticks = list(range(math.ceil(y_min - 1), math.ceil(y_max + 1.5), math.ceil(abs(y_max - y_min) / 5)))
    plt.yticks(y_ticks)

    plt.xticks(rotation=45, ha='right')
    
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title(f'{tik} Close Price for 2020')
    
    plt.grid()
    plt.legend()
    
    outputFile = f"/home/xikron/Projects/misc/graphs/{tik}_{str(graph)}.pdf"
    graph += 1
    plt.tight_layout()
    plt.savefig(outputFile, format='pdf')
    plt.close()

def getContractInterval(data, contractDate):
    return data[data['date'].dt.strftime('%Y-%m-%d').str.contains(getTradingDay(contractDate))]

from pandas.tseries.offsets import CustomBusinessDay
from pandas.tseries.holiday import USFederalHolidayCalendar

def getTradingDay(contractDate):
    us_business_day = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    adjDate = contractDate if us_business_day.is_on_offset(contractDate) else contractDate + us_business_day
    return adjDate.strftime('%Y-%m-%d')


In [167]:
lhxINTD = getYTDData("LHX", "60min", "2022-01-01")
lhxINTD.shape
genGraph("LHX", lhxINTD, lhxDf)

In [166]:
genGraph("LDOS", ldosINTD, ldosDf)

In [164]:
genGraph("RTX", rtxINTD, rtxDf)

In [118]:
lhxINTD = getYTDData("LHX", "60min", "2020-01-01")

API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called


In [153]:
genGraph("LMT", lmtINTD, lmTDf)

In [126]:
lmtINTD

,date,open,high,low,close,volume
0,2021-12-01 10:00:00,334.290,335.34,333.01,333.790,169551.0
1,2021-12-01 11:00:00,333.880,335.33,333.12,335.320,193659.0
2,2021-12-01 12:00:00,335.140,335.95,333.42,333.770,184512.0
3,2021-12-01 13:00:00,333.740,334.11,331.06,331.130,133133.0
4,2021-12-01 14:00:00,331.060,331.80,330.36,331.240,125500.0
...,...,...,...,...,...,...
1752,2023-01-31 11:00:00,460.770,461.66,459.01,461.560,68054.0
1753,2023-01-31 12:00:00,461.655,463.23,460.83,461.590,100246.0
1754,2023-01-31 13:00:00,461.590,461.75,460.26,460.580,35328.0
1755,2023-01-31 14:00:00,460.700,462.02,459.53,461.807,87463.0


In [117]:
genGraph("GD", gdINTD, gdDf)

In [120]:
genGraph("LHX", lhxINTD, lhxDf)